## To-do list
- metrics functions
- check for directories structure (model for prediction, databases, saving directory for the drawdown/VaR/etc plots, backup directory for model, backup directory for plots)
- import metrics (VaR, cVaR, drawdown, etc.) functions from metrics module
- functions module maybe?
- Binance access and etc (copy from Touring)
- Copy keys.py explanation from Touring
- Two main fund strategies: (1) passive - trade on 1st of each month, keep in basked the 10 crytpos with the most market cap; (2) active - trade every week based on predicted returns for the next week
- Let the user define how many assets to invest into
- Tem que usar sentiment analysis de twitter, reddit, google news, etc. Trabalhar com dados em T+0 e em T-1, T-2, T-3...
- Faz defasagem para RSI também
- Ver se tem algum dado de taxa de juros americana/europeia/chinesa(?)/japonesa diária
- Ou algum dado 'mundial' para ajudar na intepretação dos dados
- Também trabalhar com defasagens (1, 2, 3...) nesses dados
- When using Markovwitz, use pypfopt package


In [17]:
#------ Import basic packages
#import matplotlib.pyplot as plt
#import seaborn as sns
#import smtplib  # Needed for the e-mail reports
#import binance.enums  # Responsible for trading

#import numpy as np
import pandas as pd
#import pandas_ta
from functions import *
pd.set_option('display.float_format', lambda x: '%.8f' % x)

In [ ]:
#from metrics import mvcriterion, optim_mvcrit

#wallet = binance_wallet()

# fetch_data() checks for the most liquid cryptos and fetches the data from Binance
data = fetch_data(days=365*2, interval='1d')
#data.drop('cluster_num', axis=1, inplace=True)

# Let's calculate the technical indicators
data = estimate_indicators(data, rsi=20, bbands=20, roll=13)

# Now we need to estimate the optimal number of clusters to
# allocate our resources
clust = estimate_kmeans(data.drop('close', axis=1), cutoff=0.25)

# Sets the params for the clustering function used with apply below
params2 = {'clusters':clust,
           'type':1}

# Groups the data and applies the clustering
data = data.dropna().groupby('time', group_keys=False).apply(clustering, **params2)

In [19]:
clustering(data, clusters=clust)

close         rsi      bb_low      bb_mid  \
time       asset                                                         
2024-08-11 ADAUSDT      0.32760000 38.45415952  0.26663444  0.31636547   
           AVAXUSDT    20.40000000 38.05622382  2.96938025  3.22765840   
           BCHUSDT    330.20000000 42.52173310  5.70077901  5.90773766   
           BNBUSDT    503.40000000 42.55268154  6.13995267  6.29360873   
           BTCUSDT  58712.59000000 44.17268388 10.90012060 11.04190830   
...                            ...         ...         ...         ...   
2025-08-24 SUIUSDT      3.69690000 51.85167276  1.45270909  1.55475203   
           TRXUSDT      0.34500000 69.38981072  0.26959403  0.28384819   
           USDCUSDT     0.99990000 52.95493935  0.69284361  0.69305717   
           XLMUSDT      0.43430000 57.61482171  0.31834278  0.35002813   
           XRPUSDT      3.16010000 56.11976473  1.34833603  1.41205672   

                        bb_high         atr        macd   return_1m  \
time       asset                                                      
2024-08-11 ADAUSDT   0.36609650 -0.63959648 -0.68176289 -0.12887411   
           AVAXUSDT  3.48593654 -0.20707400 -1.04834741 -0.12011731   
           BCHUSDT   6.11469631  0.38093983 -0.93303018 -0.06362809   
           BNBUSDT   6.44726479  0.76033074 -1.74653466 -0.03874623   
           BTCUSDT  11.18369600  0.47505499 -1.17213801 -0.01729923   
...                         ...         ...         ...         ...   
2025-08-24 SUIUSDT   1.65679498  0.72330100  0.13128777 -0.07556667   
           TRXUSDT   0.29810235  0.37549642  1.29744824  0.03849026   
           USDCUSDT  0.69327073 -0.42858954  0.23721478  0.00015005   
           XLMUSDT   0.38171348  1.09809018  0.63713248 -0.01120642   
           XRPUSDT   1.47577741  1.33269835  0.60405713 -0.01223953   

                      return_2m   return_3m   return_4m   return_5m  \
time       asset                                                      
2024-08-11 ADAUSDT  -0.05842689 -0.05744469 -0.05131335 -0.06488347   
           AVAXUSDT -0.09251787 -0.08862587 -0.07219028 -0.09461084   
           BCHUSDT  -0.06301659 -0.06203317 -0.05106264 -0.03773524   
           BNBUSDT  -0.04719606 -0.02177852 -0.01746483 -0.01194884   
           BTCUSDT  -0.03130065 -0.01998814 -0.01252432 -0.01342583   
...                         ...         ...         ...         ...   
2025-08-24 SUIUSDT   0.06260330  0.01691243  0.01527402  0.06762745   
           TRXUSDT   0.05651103  0.04118621  0.04260138  0.04133448   
           USDCUSDT  0.00005001  0.00008337  0.00000000 -0.00007997   
           XLMUSDT   0.15839985  0.08418835  0.06349662  0.06678159   
           XRPUSDT   0.09400399  0.06389954  0.04895416  0.05099165   

                      return_6m  cluster_num  
time       asset                              
2024-08-11 ADAUSDT  -0.04803303            3  
           AVAXUSDT -0.04938383            3  
           BCHUSDT   0.01688143            0  
           BNBUSDT   0.02182576            4  
           BTCUSDT   0.01060909            2  
...                         ...          ...  
2025-08-24 SUIUSDT   0.04072666            3  
           TRXUSDT   0.03358293            3  
           USDCUSDT -0.00002500            3  
           XLMUSDT   0.04224743            3  
           XRPUSDT   0.03306880            3  

[408 rows x 14 columns]

In [3]:
import os
resources_dir = './resources/'
backup_dir = 'older_versions/'

    # Check for any pair file. If it exists and is newer than a month, loads it.
arqs = os.listdir(resources_dir)

    # Keeps only generated files, disregarding folders or other misc files
for i in arqs:
    if 'pairs' not in str(i):
        print(f'ESSA MERDA IMPRIME MAS NÃO REMOVE PELO JEITO, i = {i}')
        arqs.remove(i)

#print(arqs)
"""
if len(arqs) == 0:
    print('Trading file not found in folder. Creating would be here...')
    

elif len(arqs) == 1:
    arqs = arqs[0]
    print(f"Trading pairs file '{arqs}' found, checking version.")
    file_found = arqs.split('.csv')[0]
    file_found = file_found.split('_')[1]
"""

'\nif len(arqs) == 0:\n    print(\'Trading file not found in folder. Creating would be here...\')\n\n\nelif len(arqs) == 1:\n    arqs = arqs[0]\n    print(f"Trading pairs file \'{arqs}\' found, checking version.")\n    file_found = arqs.split(\'.csv\')[0]\n    file_found = file_found.split(\'_\')[1]\n'

In [ ]:
for i in data.index.get_level_values('time').unique().to_list():
    g = data.xs(i, level=0)
    plot_clusters(g, attr_1='close_norm')

### Down here are the tweaked functions:

In [3]:
#------ Indicators function
def estimate_indicators(df, rsi=20, bbands=20, roll=13):
    """
    Description: gets the data from the web and calculates its indicators

    Input:  rsi (int), time window for the RSI indicator. Defaults to 20;
            bbands (int), time window for the Bollinger Bands indicator. Defaults to 20;
            roll (int), rolling window for moving average. Defaults to 13.
    """
    import pandas as pd
    import pandas_ta
    import numpy as np

    # Close prices standardization
    print('Standardizing close prices')
    df['close_norm'] = df['close'].groupby('asset').transform(lambda x: (x - x.mean())/x.std())

    print('Calculating RSI (momentum).')
    # Calculating Relative Strenght Index (RSI) - momentum indicator
    # The RSI indicator won't be standardized for its use in clustering
    df['rsi'] = df.groupby(level=0)['close'].transform(lambda x: pandas_ta.rsi(close=x, length=rsi))
    #df.xs('BTCUSDT', level=0)['rsi'].plot() # to check if it's worked just uncomment the beginning of this line

    print('Calculating Bollinger Bands (vol).')
    # Calculating Bollinger Bands - volatility indicator (overbought/oversold)
    df['bb_low'] = df.groupby(level=0)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=bbands).iloc[:,0])
    df['bb_mid'] = df.groupby(level=0)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=bbands).iloc[:,1])
    df['bb_high'] = df.groupby(level=0)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=bbands).iloc[:,2])

    print('Calculating ATR (vol).')
    # Average True Range (ATR) - volatility indicator
    # Since this function uses 3 features to compute the indicator (high, low, close), it is needed to use 'apply' instead of 'transform',
    # and for that a custom function is needed (check it in Functions section).
    df['atr'] = df.groupby(level=0, group_keys=False).apply(atr_calc)

    print('Calculating MACD (momentum).')
    # Moving Average Convergence-Divergence (MACD) - momentum indicator
    # Same reasonig as ATR, a custom function is needed here.
    df['macd'] = df.groupby(level=0, group_keys=False).apply(macd_calc).iloc[:,0]

    print('Calculating dollar volume in millions.')
    # Dollar volume (based on closing price), divided by 1mil
    df['dollar_vol'] = df['volume']*df['close']/1e6

    print('Aggregating data to bi-weekly periods, filtering best cryptos.')
    # Aggregate to bi-weekly level and filter N most market capped cryptos
    indicators = [c for c in df.columns.unique() if c not in ['dollar_vol', 'open', 'high', 'low', 'volume']]
    p_dvol = df.unstack(level=0)['dollar_vol'].resample('2W').mean().stack('asset').to_frame('dollar_vol')
    p_indc = df.unstack(level=0)[indicators].resample('2W').last().stack('asset', future_stack=True)
    df = pd.concat([p_dvol, p_indc], axis=1).dropna()

    print('Creating dollar volume moving averages.')
    # 13-week moving average of dollar volume for each asset
    df['dollar_vol'] = df['dollar_vol'].unstack('asset').rolling(roll).mean().stack()

    print('Checking cryptos liquidity.')
    # Bi-weekly rank for each asset by dollar volume (a.k.a. liquidity), smaller rank is better (most liquid)
    df['liquidity_lvl'] = df.groupby('time')['dollar_vol'].rank(ascending=False)

    print('Creating a rank for the best cryptos in the dataset.')
    # Top 15 cryptos fortnightly, able to drop volume and liquidity features already
    mask = df['liquidity_lvl'] < 16
    df = df.loc[mask].drop(['dollar_vol', 'liquidity_lvl'], axis=1)

    print('Estimating returns.')
    df = df.groupby(level='asset', group_keys=False).apply(estimate_returns).dropna()

    print('All indicators successfuly calculated.')

    return df


#------ Function to estimate the optimal k value for clustering
def estimate_kmeans(data, max_k=10, cutoff=0.125, graph=False):
    """
    Description: estimates the optimal k value for clustering based on a cutoff value

    Input: days (pandas Series or DataFrame), the values to perform the clustering analysis;
            max_k (int), the maximum number of clusters. Defaults to 10;
            cutoff (float), the percentage (in decimals) in change to evaluate. Defaults to 0.125;
            graph (bool), whether plot a graph or not. Defaults to False

    Output: opt_k (int), the theorical optimum value of k
    """
    from sklearn.cluster import KMeans
    import pandas as pd
    
    # Creates empty lists to store mean and intertia values
    means = []
    inertias = []

    # Cycle through data to calculate means and inertias 
    for k in range(1, max_k):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(pd.DataFrame(data))

        means.append(k)
        inertias.append(kmeans.inertia_)

    # Creates a pandas DataFrame with the results
    calc = pd.DataFrame([means, inertias], index=['Means', 'Inertia']).T

    # Checks which inertias are under the cutoff value and defines the best k under this assumption
    try:
        mask = (calc.Inertia / calc.Inertia[0]) < cutoff
        opt_k = int(calc[mask].Means.min())
    
        # Plot the elbow graph if graph param is set to True
        if graph == True:
            import matplotlib.pyplot as plt

            if cutoff != 0.125:
                print(f'Theoretical best k: {opt_k}, with change cutoff {cutoff}')
            else:
                print(f'Theoretical best k: {opt_k}, with default change cutoff value {cutoff}')

            fig = plt.subplots(figsize=(10,5))
            plt.plot(means, inertias, 'o-')
            plt.xlabel('Number of Clusters (k)')
            plt.ylabel('Inertia')
            plt.grid(True)
            plt.show()
            print(f'WARNING! Clustering not registered in the dataset. For it to be done, use hyperparameter "graph=False".')

        else:
            return opt_k

    except ValueError as e:
        return print(f'ERROR! Hyperparameter "cutoff" too low, try to use a greater value')

    

#------ Clustering itself
def clustering(df, metric='rsi', clusters=None, init=0):
    """
    Description: performs the clustering of the data, based on estimate_kmeans() results.

    Input: df (pandas DataFrame), the data to perform the clustering;
            metric (str), metric to estimate the centroids. Defaults to 'rsi';
            clusters (int), either user-defined or the results from estimate_kmeans();
            init (int), 0 (k-means++), 1 (centroids) or 2 (random). Defaults to 0.

    Output: df (pandas DataFrame), the clustered DataFrame
    """
    import pandas as pd
    from sklearn.cluster import KMeans
    import numpy as np
    
    temp = pd.Series(df.columns == metric)
    num = temp[temp.index == True].index[0]
    feat = int(num)

    # Set target values
    target_values = []
    for num in range(1, clusters+1):
        temp = int(num * np.quantile(range(0, 101), 1/(clusters+1)))
        target_values.append(temp)    

    # Calculate centroids
    centroids = np.zeros((len(target_values), len(df.columns)))
    centroids[:, feat] = target_values
    
    if init == 0:
        df['cluster_num'] = KMeans(n_clusters=clusters,
                                   random_state=1,
                                   init='k-means++').fit(df).labels_
        return df
    
    elif init == 1:
        df['cluster_num'] = KMeans(n_clusters=clusters,
                                   random_state=1,
                                   init=centroids).fit(df).labels_
        return df

    elif init == 2:
        df['cluster_num'] = KMeans(n_clusters=clusters,
                                   random_state=1,
                                   init='random').fit(df).labels_
        return df
    
    else:
        print('"init" argument MUST be 0 (k-means++), 1 (centroids) or 2 (random)')
    